In [1]:
import cv2
import numpy as np
from mtcnn import MTCNN
import os
from keras_facenet import FaceNet
from sklearn.preprocessing import Normalizer, LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
from datetime import datetime
from openpyxl import Workbook

ModuleNotFoundError: No module named 'mtcnn'

### Tạo folder

In [ ]:
raw_directory = "raw"
processed_directory = "processed"

# check if raw directory exists
if os.path.exists(raw_directory):
    print(f"{raw_directory} already exists.")
else:
    print(f"{raw_directory} is not exist")
    # create directory
    os.mkdir(raw_directory)
    print(f"{raw_directory} created successfully.")

# check if processed directory exists
if os.path.exists(processed_directory):
    print(f"{processed_directory} already exists.")
else:
    print(f"{processed_directory} is not exist")
    # create directory
    os.mkdir(processed_directory)
    print(f"{processed_directory} created successfully.")

raw already exists.
processed already exists.


### nhập tên các thứ đồ

In [ ]:
wrong_input = '\/:*?"<>|'
while True:
    input_valid = True
    train_dir = input('input name')
    for char in train_dir:
        if char in wrong_input:
            input_valid = False
            print('Input contains invalid characters: \/:*?"<>|')
            break
            
    # phần trên là check input có được tạo thành folder hay không
    if input_valid == False:
        continue
    print("Input is valid.")
    # phần dưới là check folder đấy đã có chưa, có muốn thêm file/vid vào folder đấy hay không
    if os.path.exists("processed/" + train_dir) and os.path.exists("raw/" + train_dir):
        print(f"processed/{train_dir} and raw/{train_dir} already exists.")
        merge = input('input on the same folder(y/n):')
        if merge == 'y':
            break
        else:
            continue
    else:
        print(f"processed/{train_dir} is not exist.")
        # create directory
        os.mkdir("processed/" + train_dir)
        print(f"raw/{train_dir} is not exist.")
        os.mkdir("raw/" + train_dir)
        print(f"{train_dir} created successfully.")
        break


Input is valid.
processed/HE171281 is not exist.
raw/HE171281 is not exist.
HE171281 created successfully.


### quay vid/ lưu vids

In [ ]:
# define a video capture object
vid = cv2.VideoCapture(0)
   
# We need to check if camera
# is opened previously or not
if (vid.isOpened() == False): 
    print("Error reading video file")
frame_width = int(vid.get(3))
frame_height = int(vid.get(4))

size = (frame_width, frame_height)

result = cv2.VideoWriter(f'raw/{train_dir}/rawVid.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)
   

while(True):
	
	# Capture the video frame
	# by frame
	ret, frame = vid.read()
	
	result.write(frame)

	# Display the resulting frame
	cv2.imshow('frame', frame)
	
	# the 'q' button is set as the
	# quitting button you may use any
	# desired button of your choice
	if cv2.waitKey(1) & 0xFF == ord('q'):
		break

# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

### đưa từ vid thành ảnh

In [ ]:
video = cv2.VideoCapture(f'raw/{train_dir}/rawVid.avi')
frames = []
interval = 5  #muốn cách bao nhiêu ảnh
frame_count = 0 
while True:
    # Read a frame from the video
    ret, frame = video.read()

    # Check if the frame was successfully read
    if not ret:
        # If the frame was not successfully read, break out of the loop
        break
    if frame_count % interval == 0:
        frames.append(frame)

    frame_count += 1
    # Add the frame to the list of frames

# Release the video file
video.release()

### xử lý ảnh

In [ ]:


detector = MTCNN()
num_of_frame = 0
for frame in frames:
    num_of_frame += 1
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = detector.detect_faces(image)

    print(result)


    if len(result) > 0:
        bounding_box = result[0]['box']
        keypoints = result[0]['keypoints']

        crop_img = image[bounding_box[1]:bounding_box[1] + bounding_box[3], bounding_box[0]:bounding_box[0]+bounding_box[2]]
        resized_image = cv2.resize(crop_img, (160, 160))
        save_img = cv2.cvtColor(resized_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(f'processed/{train_dir}/frame {num_of_frame}.jpg', save_img)


NameError: name 'frames' is not defined

### Tạo model

In [ ]:
faces = []
Id = []
IDUnique = []
for subdir in os.listdir('processed'): #liệt kê mọi folder trong folder directory
    if os.path.isdir(os.path.join('processed', subdir)):
        IDUnique.append(subdir) # append mọi tên folder 
for id in IDUnique:
    for facedir in os.listdir(os.path.join('processed', id)):
        face = cv2.imread(os.path.join('processed', id, facedir))
        faces.append(face)
        Id.append(id)


In [ ]:
embedder = FaceNet()
embeddings = embedder.embeddings(faces)


5/5 [==============================] - 4s 385ms/step


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(embeddings, Id, test_size=.1, random_state=42)

In [ ]:
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(X_train)
testX = in_encoder.transform(X_test)
out_encoder = LabelEncoder()
out_encoder.fit(y_train)
trainy = out_encoder.transform(y_train)
testy = out_encoder.transform(y_test)
model = SVC(kernel='linear')
model.fit(trainX, trainy)



SVC(kernel='linear')

In [ ]:
# predict
yhat_train = model.predict(trainX)
yhat_test = model.predict(testX)
# score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

Accuracy: train=100.000, test=93.333


In [ ]:
filename = 'svm_saved.sav'
pickle.dump(model, open(filename, 'wb'))
encoder_file_name = 'out_encoder.pkl'
pickle.dump(out_encoder, open(encoder_file_name, 'wb'))


### Triển khai models

In [ ]:
import cv2
import numpy as np
from mtcnn import MTCNN
import os
from keras_facenet import FaceNet
from sklearn.preprocessing import Normalizer, LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
from datetime import datetime
from openpyxl import Workbook


vid = cv2.VideoCapture(0)
detector = MTCNN()
embedder = FaceNet()
out_encoder = LabelEncoder()
in_encoder = Normalizer(norm='l2')
filename = 'svm_saved.sav'
encoder_file_name = 'out_encoder.pkl'
loaded_model = pickle.load(open(filename, 'rb'))
loaded_out_encoder = pickle.load(open(encoder_file_name, 'rb'))
wb = Workbook()
sheet = wb.active
sheet['A1'] = 'Name'
sheet['B1'] = 'Time'
# We need to check if camera
# is opened previously or not
if (vid.isOpened() == False): 
    print("Error reading video file")
frame_width = int(vid.get(3))
frame_height = int(vid.get(4))

size = (frame_width, frame_height)

#result = cv2.VideoWriter(f'final.avi', cv2.VideoWriter_fourcc(*'MJPG'),10, size)

interval = 25  #muốn cách bao nhiêu ảnh
frame_count = 0 
attended = set()
while(True):
	# Capture the video frame
	# by frame
	ret, frame = vid.read()
	if not ret:
        # If the frame was not successfully read, break out of the loop
		break

	#result.write(frame)

	# Display the resulting frame
	cv2.imshow('frame', frame)
	if frame_count % interval == 0:
		image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
		result = detector.detect_faces(image)
		if len(result) > 0:
			for each_result in result:
				bounding_box = each_result['box']
				keypoints = each_result['keypoints']
				crop_img = image[bounding_box[1]:bounding_box[1] + bounding_box[3], bounding_box[0]:bounding_box[0]+bounding_box[2]]
				resized_image = cv2.resize(crop_img, (160, 160))
				save_img = cv2.cvtColor(resized_image, cv2.COLOR_RGB2BGR)
				embeddings = embedder.embeddings([resized_image])
				testX = in_encoder.transform(embeddings)
				y_pred_encoded = loaded_model.predict(testX)
				y_pred = loaded_out_encoder.inverse_transform(y_pred_encoded)
				time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
				if y_pred[0] in attended:
					continue
				else:
					attended.add(y_pred[0])
					sheet.append([y_pred[0], time])
	frame_count += 1
	
	# the 'q' button is set as the
	# quitting button you may use any
	# desired button of your choice
	if cv2.waitKey(1) & 0xFF == ord('q'):
		wb.save(filename="attend list.xlsx")
		break

# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

1/1 [==============================] - 0s 72ms/step


In [ ]:
from openpyxl import Workbook
wb = Workbook()
sheet = wb.active
sheet['A1'] = 'Name'
sheet['B1'] = 'Time'
sheet.append(['a', 1])
wb.save(filename="hello_world.xlsx")


In [ ]:
attended = set()
attended.add('hoang')
attended.add('hoang')
attended

{'hoang'}